## Collect the data from Youtube

### Scrap music video list & number of views

With selenium and firefox,
 * Scrap kpop music videos on youtube
 * Scrap # of views for certain video

#### import prerequisites

In [6]:
from selenium.webdriver import Firefox
from pymongo import MongoClient 
from datetime import datetime

import random
import time
import os

#### connect to mongodb set up on the cloud

with SSH tunnel through 47017 port

In [11]:
connection = MongoClient(port=47017)
db = connection['youtube_scrap']

In [3]:
# Initialize browser
browser = Firefox()

#### Collect video list

In [13]:
def collect_videos(browser, db):    
    """Collect video list from youtube and save to videos collection"""
    url = 'https://www.youtube.com/results?search_query=kpop+music+video&sp=CAMSBggEEAEYAQ%253D%253D'
    browser.get(url)
    time.sleep(2)

    for _ in range(10):
        browser.execute_script("window.scrollTo(0, window.scrollY + 200000)")
        time.sleep(3)

    # Get video title & URL
    videolinks = browser.find_elements_by_id('video-title')
    videos = [(link.text, link.get_attribute('href')) for link in videolinks]
    
    # Store to 'videos' collection
    i = 0
    coll = db['videos']
    for title, url in videos:
        i +=1
        video_id = url.split('=')[-1]
        coll.insert_one({
            'video_id': video_id,
            'title': title,
            'url': url
        })

    print(f'{i} videos were collected.')
    

In [14]:
collect_videos(browser, db)

217 videos were collected.


#### Get number of views

In [4]:
def get_view_count(browser, url):
    """Return the view_count and timestamp"""
    browser.get(url)
    time.sleep(3)
    now = datetime.now()
    sel = 'span.view-count'
    view_count = browser.find_element_by_css_selector(sel).text

    return {
        'view_count': int(''.join([n for n in view_count if n.isdigit()])),
        'timestamp': now }

In [15]:
# Get video list from videos collection
coll = db['videos']
cur = coll.find({})

In [17]:
# Store it to list
videos = [video for video in cur]

In [19]:
videos[0]

{'_id': ObjectId('5d1269888771f921313fed50'),
 'video_id': 'oDJ4ct59NC4',
 'title': "SOMI (전소미) - 'BIRTHDAY' M/V",
 'url': 'https://www.youtube.com/watch?v=oDJ4ct59NC4'}

In [6]:
# Collect number of views
while True:
    coll = db['view_count']
    for video in videos:
        print(video['title'])
        count = get_view_count(browser, video['url'])
        coll.insert_one({
            'title': video['title'],
            'view_count': count['view_count'],
            'timestamp': count['timestamp']
        })
    time.sleep(600)


Audiofreq - Rewind (Official Video) 7664
Ully Moch & Ifan Seventeen - HUN (Official Music Video NAGASWARA) #music 327731
OLD TOWN ROAD Kids Dance Music Video 42069
Ayu Ting Ting x Keremcem - Apalah Cinta (Official Music Video) 7368888
Squeeze by Fille ft Voltage Music (Official Video) 11617
DAVIS - Alright (Official Music Video) 775
MC Klinton - SHQIPE TRAPKING prod.OGRruga (Official Video HD) 6563
Hez Hazmi - Biar Aku Pergi (Official Music Video) 92296
VRKINGS - SNAKEBITE FT. HONEY CRUX | BHAGUMIA | OFFICIAL MUSIC VIDEO | LATEST HINDI RAP | 2019 942
Hard Driver - Young Birds (Official Video) 52255
Julz West - On My Own (Dance Music Video) 253
Sule & Baby Shima - Terpisah Jarak Dan Waktu (Official Music Video NAGASWARA) #music 1613066
ALBUM SONIYA VE Official || ADITYA SINGH || Music Video Song || 2019 album 2219
Agung Pradanta - Kuat Ati ( Official Music Video ) 3752
BANNED KPOP Music Videos for Stupid Reasons 503555
MOUSTO CAMARA | Syli De Guinée | 🇬🇳Official Video 2019 | By Dj.IKK 3

In [ ]:
browser.close()